In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

2022-12-06 18:39:31.571852: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from IPython.display import HTML

In [5]:
def createModel(input_shape = (203, 23*7)):
    
    X = tfl.Input(input_shape)  # define the input to the model
    lstm = tfl.LSTM(100, activation='tanh', recurrent_activation='tanh', return_sequences=True)(X)
    drop = tfl.Dropout(0.2)(lstm)
    d3 = tfl.Dense(3,activation=None)(drop)
    permute = tfl.Permute((2,1))(d3)    # change input from (None, 203, 3) to (None, 3, 203)
    
    # have layer (batch_size, 3). Want to take (b, [0,1]) and turn them into probabilities, and keep (b, [2]) as time
    # https://datascience.stackexchange.com/questions/86740/how-to-slice-an-input-in-keras
    probs = tfl.Cropping1D(cropping=(0,1))(permute) # shape (None, 2, 203)
    probs = tfl.Softmax(axis=1)(probs)
    
    time = tfl.Cropping1D(cropping=(2,0))(permute) # shape (None, 1, 203)
    
    # concatenate the probabilities and predicted_time_to_sack back into one layer
    out = tfl.Concatenate(axis=1)([probs, time]) # shape (None, 3, 203)
    out = tfl.Permute((2,1))(out) # shape (None, 203, 3)
    
    model = Model(inputs=X, outputs=out)        # create model
    
    return model

model = createModel()

2022-12-06 18:40:10.407106: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model_string = f"./rnn_model1/weights/weights_epochs10"

model.load_weights(model_string)

In [9]:
MAX_PLAY_LENGTH = 203

x_val = np.load("./seq_data/x_val.npy")
y_val = np.load("./seq_data/y_val.npy")
train_mu = np.load("./seq_data/train_mu.npy")
train_std = np.load("./seq_data/train_std.npy")


In [10]:
x_val_input = x_val.reshape(-1, MAX_PLAY_LENGTH, 23, 11)[:,:,:,4:].reshape(-1,MAX_PLAY_LENGTH,23*7)

In [59]:
x_val[0][-1]

array([ 2.02110170e+09,  2.32700000e+03,  2.80000000e+01,  4.32900000e+04,
        1.00000000e+00,  3.08189874e+00,  2.72795400e+01,  2.51652325e-01,
        1.54218847e+00, -1.56078303e-01,  2.91133180e+00,  2.27122355e+00,
        2.11556175e+00,  2.22141293e+00,  2.59839161e+00,  2.90329814e+00,
        2.43292136e+00,  5.21522985e+00,  3.75868070e+00,  4.90316604e+00,
        3.11618021e+00,  3.51657135e+00,  2.27122355e+00,  2.11556175e+00,
        2.22141293e+00,  2.49403962e+00,  3.00990694e+00,  2.41502604e+00,
        1.35101790e+00,  1.65503874e+00,  4.99064482e-01,  2.41492950e+00,
        3.10048793e+00,  2.27122355e+00,  2.11556175e+00,  2.22141293e+00,
        2.42682690e+00,  1.32100520e+00,  3.06011713e+00,  1.32380887e+00,
        1.90062185e+00,  2.28180766e+00,  4.00283831e+00,  1.94254024e+00,
        2.27122355e+00,  2.11556175e+00,  2.22141293e+00,  2.37355049e+00,
        3.00450381e+00,  3.03113621e+00,  1.19796790e+00,  2.31376168e+00,
        1.30823188e+00,  

In [11]:
preds = model.predict(x_val_input)

41/41 [==============================] - 4s 70ms/step


In [12]:
print(x_val.shape)
print(x_val_input.shape)
print(preds.shape)

(1300, 203, 253)
(1300, 203, 161)
(1300, 203, 3)


In [16]:
# unadjust coordinates
unadj_x_val = x_val
unadj_x_val[:,:,5:] = (x_val[:,:,5:]*train_std[5:])+train_mu[5:]
unadj_x_val[:,:,6] += (53.3/2)
        

In [25]:
four_dim = unadj_x_val.reshape(unadj_x_val.shape[0], MAX_PLAY_LENGTH, 23, -1)

In [37]:
four_dim.shape

(1300, 203, 23, 11)

In [38]:
preds_repeat = np.repeat(preds, 23, axis=2).reshape(-1, MAX_PLAY_LENGTH, 23, 3)

preds_repeat.shape

(1300, 203, 23, 3)

In [39]:
val_with_preds = np.concatenate((four_dim, preds_repeat), axis=3)

val_with_preds.shape

(1300, 203, 23, 14)

In [46]:
test = val_with_preds[0]

tester = test.reshape(-1, 14)

In [48]:
tester[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       -0.40329598, 26.64145936, -0.34554969, -0.3554946 , -0.35967242,
       -0.38184469,  0.41544172,  0.41544172,  0.41544172])

In [14]:
col_names = ['gameId', 'playId', 'frameId', 'nflId', 'team_indicator', 'adj_x', 'adj_y', 's', 'a', 'adj_o', 'adj_dir', 'no_sack_prob', 'sack_prob', 'pred_time']

val_with_preds_df = pd.DataFrame(val_with_preds.reshape(-1, len(col_names)), columns=col_names)
val_with_preds_df.gameId.astype('int64', copy=False)


In [15]:
val_with_preds_df.head()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time
0,2.021102e+09,521.0,6.0,38544.0,1.0,86.66,35.35,0.00,7.000000e-02,170.22,156.66,1.0,4.266793e-08,-1.107836
1,2.021102e+09,521.0,6.0,38553.0,2.0,88.47,33.00,0.00,2.220446e-16,327.67,261.74,1.0,4.266793e-08,-1.107836
2,2.021102e+09,521.0,6.0,40171.0,1.0,86.74,32.14,0.02,3.700000e-01,174.35,153.68,1.0,4.266793e-08,-1.107836
3,2.021102e+09,521.0,6.0,41619.0,2.0,87.92,29.86,0.01,1.200000e-01,338.62,339.74,1.0,4.266793e-08,-1.107836
4,2.021102e+09,521.0,6.0,42444.0,1.0,86.47,25.06,0.01,1.900000e-01,145.39,156.39,1.0,4.266793e-08,-1.107836


In [16]:
val_with_preds_df['true_sack'] = np.repeat(y_val[:,1], 23)

In [17]:
val_with_preds_df.head()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
0,2.021102e+09,521.0,6.0,38544.0,1.0,86.66,35.35,0.00,7.000000e-02,170.22,156.66,1.0,4.266793e-08,-1.107836,0.0
1,2.021102e+09,521.0,6.0,38553.0,2.0,88.47,33.00,0.00,2.220446e-16,327.67,261.74,1.0,4.266793e-08,-1.107836,0.0
2,2.021102e+09,521.0,6.0,40171.0,1.0,86.74,32.14,0.02,3.700000e-01,174.35,153.68,1.0,4.266793e-08,-1.107836,0.0
3,2.021102e+09,521.0,6.0,41619.0,2.0,87.92,29.86,0.01,1.200000e-01,338.62,339.74,1.0,4.266793e-08,-1.107836,0.0
4,2.021102e+09,521.0,6.0,42444.0,1.0,86.47,25.06,0.01,1.900000e-01,145.39,156.39,1.0,4.266793e-08,-1.107836,0.0


In [30]:
true_sack_df = val_with_preds_df.query("true_sack == 1")
true_sack_df.gameId = true_sack_df.gameId.astype('int64')

/var/folders/59/2qy07m9n56j1hjrbkcj93rhc0000gn/T/ipykernel_9862/1917819778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_sack_df.gameId = true_sack_df.gameId.astype('int64')


In [31]:
true_sack_df.head()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
4278,2021101704,746.0,6.0,43290.0,1.0,49.60,23.64,0.00,2.220446e-16,3.12,201.92,1.0,1.022863e-07,-1.036884,1.0
4279,2021101704,746.0,6.0,43299.0,2.0,42.90,33.98,0.72,2.500000e-01,227.58,253.04,1.0,1.022863e-07,-1.036884,1.0
4280,2021101704,746.0,6.0,43350.0,2.0,38.00,20.17,1.60,3.200000e-01,161.11,144.22,1.0,1.022863e-07,-1.036884,1.0
4281,2021101704,746.0,6.0,43453.0,1.0,49.31,25.25,0.00,2.220446e-16,3.34,175.39,1.0,1.022863e-07,-1.036884,1.0
4282,2021101704,746.0,6.0,43455.0,2.0,47.37,22.56,0.00,1.100000e-01,167.96,81.33,1.0,1.022863e-07,-1.036884,1.0


In [32]:
test_game_id = 2021101704
test_play_id = 746.0

In [33]:

play_df = val_with_preds_df.query("gameId == @test_game_id and playId == @test_play_id")
print(f"play_df shape = {play_df.shape}")

play_df.head()

play_df shape = (1150, 15)


,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
4278,2.021102e+09,746.0,6.0,43290.0,1.0,49.60,23.64,0.00,2.220446e-16,3.12,201.92,1.0,1.022863e-07,-1.036884,1.0
4279,2.021102e+09,746.0,6.0,43299.0,2.0,42.90,33.98,0.72,2.500000e-01,227.58,253.04,1.0,1.022863e-07,-1.036884,1.0
4280,2.021102e+09,746.0,6.0,43350.0,2.0,38.00,20.17,1.60,3.200000e-01,161.11,144.22,1.0,1.022863e-07,-1.036884,1.0
4281,2.021102e+09,746.0,6.0,43453.0,1.0,49.31,25.25,0.00,2.220446e-16,3.34,175.39,1.0,1.022863e-07,-1.036884,1.0
4282,2.021102e+09,746.0,6.0,43455.0,2.0,47.37,22.56,0.00,1.100000e-01,167.96,81.33,1.0,1.022863e-07,-1.036884,1.0


In [34]:
play_df.tail()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
5423,2.021102e+09,746.0,55.0,52473.0,2.0,37.26,32.93,2.03,3.96,176.78,81.67,0.999771,0.000229,-0.878134,1.0
5424,2.021102e+09,746.0,55.0,52483.0,1.0,53.47,27.26,2.55,1.05,73.29,114.98,0.999771,0.000229,-0.878134,1.0
5425,2.021102e+09,746.0,55.0,53436.0,1.0,55.38,24.77,2.45,0.84,115.84,141.15,0.999771,0.000229,-0.878134,1.0
5426,2.021102e+09,746.0,55.0,53541.0,1.0,21.52,30.69,7.23,3.77,93.89,40.03,0.999771,0.000229,-0.878134,1.0
5427,2.021102e+09,746.0,55.0,0.0,3.0,58.90,28.74,2.36,3.53,0.00,0.00,0.999771,0.000229,-0.878134,1.0


In [35]:
animated_play = AnimateFeature(play_df)
HTML(animated_play.ani.to_jshtml())

In [21]:
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.patches as patches

In [22]:
class AnimateFeature():
    
    # initialize variables we need. 
    # Example: Need to initialize the figure that the animation command will return
    def __init__(self, play_df, displayNumbers=False) -> None:
        
        self.MAX_FIELD_PLAYERS = 22
        self.start_x = 0
        self.stop_x = 120
        self.start_y = 53.3
        self.play_df = play_df
        self.frames_list = play_df.frameId.unique()
        self.games_df = pd.read_csv("data/games.csv")
        self.predictions = True
        self.displayNumbers = displayNumbers
        self.FIG_HEIGHT = 4
        self.FIG_WIDTH = 8
        
        fig, ax = plt.subplots(1, figsize=(self.FIG_WIDTH, self.FIG_HEIGHT))
        
        self.fig = fig
        self.field_ax = ax
        
        # create new axis for home, away, jersey
        self.ax_home = self.field_ax.twinx()
        self.ax_away = self.field_ax.twinx()
        self.ax_jersey = self.field_ax.twinx()
        self.ax_pred = self.field_ax.twinx()
        
        self.ani = animation.FuncAnimation(self.fig, self.update, frames=len(self.frames_list),
                                          init_func=self.setup_plot, blit=False)
        
        plt.close()
        
    # initialization function for animation call
    def setup_plot(self):

        endzones = True
        linenumbers = True
        
        # set axis limits
        self.set_axis_plots(self.field_ax, self.stop_x, self.start_y)
        self.set_axis_plots(self.ax_home, self.stop_x, self.start_y)
        self.set_axis_plots(self.ax_away, self.stop_x, self.start_y)
        self.set_axis_plots(self.ax_jersey, self.stop_x, self.start_y)
        self.set_axis_plots(self.ax_pred, self.stop_x, self.start_y)

        # set up colors and patches for field
        self.set_up_field(endzones, linenumbers)
        
        # create scatterplots on axis for data
        self.scat_field = self.field_ax.scatter([], [], s=50, color='orange')
        self.scat_home = self.ax_home.scatter([], [], s=50, color='blue')
        self.scat_away = self.ax_away.scatter([], [], s=50, color='red')
        
        # create box for prediction
        self.scat_pred = self.ax_pred.text(0, 0, '', fontsize = self.FIG_WIDTH, 
                                           bbox=dict(boxstyle="square", facecolor="white"),
                                           horizontalalignment = 'left', verticalalignment = 'top', c = 'black')
        
        # add direction stats and jersey numbers/names
        self._scat_jersey_list = []
        self._scat_number_list = []
        self._scat_name_list = []
        self._a_dir_list = []
        self._a_or_list = []
        for _ in range(self.MAX_FIELD_PLAYERS):
            self._scat_jersey_list.append(self.ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'white'))
            self._scat_number_list.append(self.ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black'))
            self._scat_name_list.append(self.ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black'))
            
            self._a_dir_list.append(self.field_ax.add_patch(patches.Arrow(0, 0, 0, 0, color = 'k')))
            self._a_or_list.append(self.field_ax.add_patch(patches.Arrow(0, 0, 0, 0, color = 'k')))
        
        # return all axis plots that you want to update
        return (self.scat_field, self.scat_home, self.scat_away, self.ax_pred, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)
    
    def update(self, i):
        frame = self.frames_list[i]
        time_df = self.play_df.query("frameId == @frame")
        
        #time_df['team_indicator'] = self.add_team_indicator(time_df)
        #print(time_df)
        
        label_list = time_df.team_indicator.unique()
        #print(label_list)
        label1= label_list[0]
        label2 = label_list[1]
        label3 = label_list[2]
        
        # update each team/football x,y coordinates
        for label in label_list:
            label_data = time_df[time_df.team_indicator == label]
            
            if label == label1:
                self.scat_field.set_offsets(label_data[['adj_x','adj_y']].to_numpy())
            elif label == label2:
                self.scat_home.set_offsets(label_data[['adj_x','adj_y']].to_numpy())
            elif label == label3:
                self.scat_away.set_offsets(label_data[['adj_x','adj_y']].to_numpy())
                
        # add prediction
        if self.predictions != None:
            sack_prob = np.round(time_df.sack_prob.values[0], 3)
            time = np.round(time_df.pred_time.values[0], 3)
            true_sack = time_df.true_sack.values[0]
            set_str = f"Sack = {true_sack}, P(sack) = {sack_prob}, time = {time}"
            self.scat_pred.set_text(set_str)
        
        #add direction and jersey info
        jersey_df = time_df[time_df.nflId != 0].reset_index()
        
        for (index, row) in jersey_df.iterrows():
            #self._scat_jersey_list[index].set_position((row.x, row.y))
            #self._scat_jersey_list[index].set_text(row.position)
            
            if self.displayNumbers:
                self._scat_number_list[index].set_position((row.adj_x, row.adj_y))
                self._scat_number_list[index].set_text(int(row.nflId))
            #self._scat_name_list[index].set_position((row.x, row.y-1.9))
            #self._scat_name_list[index].set_text(row.displayName.split()[-1])
            
            player_orientation_rad = self.deg_to_rad(self.convert_orientation(row.adj_o))
            player_direction_rad = self.deg_to_rad(self.convert_orientation(row.adj_dir))
            player_speed = row.s
            
            player_vel = np.array([np.real(self.polar_to_z(player_speed, player_direction_rad)), np.imag(self.polar_to_z(player_speed, player_direction_rad))])
            player_orient = np.array([np.real(self.polar_to_z(2, player_orientation_rad)), np.imag(self.polar_to_z(2, player_orientation_rad))])
            
            self._a_dir_list[index].remove()
            self._a_dir_list[index] = self.field_ax.add_patch(patches.Arrow(row.adj_x, row.adj_y, player_vel[0], player_vel[1], color = 'k'))
            
            self._a_or_list[index].remove()
            self._a_or_list[index] = self.field_ax.add_patch(patches.Arrow(row.adj_x, row.adj_y, player_orient[0], player_orient[1], color = 'grey', width = 2))
                

        return (self.scat_field, self.scat_home, self.scat_away, self.ax_pred, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)
    
    def set_up_field(self, endzones=True, linenumbers=True) -> None:
        yard_numbers_size = self.fig.get_size_inches()[0]*1.5

        # color field 
        rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                                    edgecolor='r', facecolor='darkgreen', zorder=0)
        self.field_ax.add_patch(rect)

        # plot
        self.field_ax.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
                    80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
                    [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
                    53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
                    color='white')

        # Endzones
        if endzones:
            ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                    linewidth=0.1,
                                    edgecolor='r',
                                    facecolor='blue',
                                    alpha=0.2,
                                    zorder=0)
            ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                    linewidth=0.1,
                                    edgecolor='r',
                                    facecolor='blue',
                                    alpha=0.2,
                                    zorder=0)
            self.field_ax.add_patch(ez1)
            self.field_ax.add_patch(ez2)
            
        if endzones:
            hash_range = range(11, 110)
        else:
            hash_range = range(1, 120)

        # add hashes
        for x in hash_range:
            self.field_ax.plot([x, x], [0.4, 0.7], color='white')
            self.field_ax.plot([x, x], [53.0, 52.5], color='white')
            self.field_ax.plot([x, x], [22.91, 23.57], color='white')
            self.field_ax.plot([x, x], [29.73, 30.39], color='white')
            
        # add linenumbers
        if linenumbers:
                for x in range(20, 110, 10):
                    numb = x
                    if x > 50:
                        numb = 120 - x
                    self.field_ax.text(x, 5, str(numb - 10),
                            horizontalalignment='center',
                            fontsize=yard_numbers_size,  # fontname='Arial',
                            color='white')
                    self.field_ax.text(x - 0.95, 53.3 - 5, str(numb - 10),
                            horizontalalignment='center',
                            fontsize=yard_numbers_size,  # fontname='Arial',
                            color='white', rotation=180)

        self.field_ax.set_xlim(self.start_x, self.stop_x)
        self.field_ax.set_ylim(0, self.start_y)
        self.field_ax.set_xticks(range(self.start_x,self.stop_x, 10))

    @staticmethod
    def set_axis_plots(ax, max_x, max_y) -> None:
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

        ax.set_xlim([0, max_x])
        ax.set_ylim([0, max_y])
        
    @staticmethod
    def convert_orientation(x):
        return (-x + 90)%360
    
    @staticmethod
    def polar_to_z(r, theta):
        return r * np.exp( 1j * theta)
    
    @staticmethod
    def deg_to_rad(deg):
        return deg*np.pi/180
        

In [48]:
# from importlib import reload

# def importOrReload(module_name, *names):
#     import sys

#     if module_name in sys.modules:
#         reload(sys.modules[module_name])
#     else:
#         __import__(module_name, fromlist=names)

#     for name in names:
#         globals()[name] = getattr(sys.modules[module_name], name)
        
# importOrReload("temp", "AnimatePlay")
